In [1]:
import os
import sys
import datetime
import pathlib
import pandas as pd
dir_path = pathlib.Path(os.path.abspath('')).resolve()
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *
print(dir_path)

/mnt/OpenFormat/python/wide_table


In [3]:
num_rows_list = [100000] # 100000
num_cols_list = [10,50, 100, 150, 200, 400, 600, 800, 1000]

In [3]:
%%time
# generate csv files
for c in num_cols_list:
    os.system(f'python3 gen_table.py {c}&')

CPU times: user 2.04 ms, sys: 591 µs, total: 2.63 ms
Wall time: 22.3 ms


In [4]:
# generate parquet and orc files 
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_default.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_default.json')
pq_name_suffix = '_' + "_".join([str(i) for i in list(pq_config[0].values())]) + '.parquet'
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
for num_rows in num_rows_list:
    for num_cols in num_cols_list:
        print(f'begin num_cols: {num_cols}')
        base_name = f'idx_run2_max9_c{num_cols}'
        os.system(f'cp {base_name}.csv {PROJ_SRC_DIR}/python/data_gen/')
        
        os.chdir(f'{PROJ_SRC_DIR}/python')
        os.system(f'python3 scripts/parquet_exp.py {base_name} pq_default -r=false -o=false')
        os.system(f'python3 scripts/orc_exp.py {base_name} orc_default')
        os.chdir(f'{dir_path}')
        os.system(f'mv ../{base_name}{orc_name_suffix} ./{base_name}.orc')
        os.system(f'mv ../{base_name}{pq_name_suffix} ./{base_name}.parquet')

loading config from /mnt/OpenFormat/python/experiments/pq_default.json ...
loading config from /mnt/OpenFormat/python/experiments/orc_default.json ...
begin num_cols: 10
loading config from ./experiments/pq_default.json ...
----Running experiment: 1 out of 1----
SNAPPY_True_1048576_1048576_2.6_2.0
write parquet time (s): 0.4850349426269531
loading config from ./experiments/orc_default.json ...
----Running experiment:----
SNAPPY_speed_0.8
write orc time (s): 1.5773646831512451
begin num_cols: 50
loading config from ./experiments/pq_default.json ...
----Running experiment: 1 out of 1----
SNAPPY_True_1048576_1048576_2.6_2.0
write parquet time (s): 2.3670949935913086
loading config from ./experiments/orc_default.json ...
----Running experiment:----
SNAPPY_speed_0.8
write orc time (s): 7.900534629821777
begin num_cols: 100
loading config from ./experiments/pq_default.json ...
----Running experiment: 1 out of 1----
SNAPPY_True_1048576_1048576_2.6_2.0
write parquet time (s): 4.705190896987915

In [4]:
# get avg time for decoding first 3 col independently
!pwd
os.system('rm -f outputs/stats.json')
output_stats = {}
for i in range(5):
    for fmt in ['parquet', 'orc']:
        for num_rows in num_rows_list:
            for num_cols in num_cols_list:
                base_name = f'idx_run2_max9_c{num_cols}'
                for col_idx in range(3):
                    dtype = 'string'
                    os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                    if fmt == 'parquet':
                        output_stats['time'] = os.popen(f"parquet-test-single-col \
                            -f {base_name}.parquet -i {col_idx}").read().split('\n')[0].split(' ')[-2]
                        output_stats['time_preload'] = os.popen(f"parquet-test-single-col \
                            -f {base_name}.parquet -i {col_idx}").read().split('\n')[0].split(' ')[-2]
                    else:
                        output_stats['time'] = float(os.popen(f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                        {base_name}.orc f{col_idx} {dtype} none \
                            0 0 one").read().split('\n')[0].split(' ')[-1])*1000
                        output_stats['time_preload'] = float(os.popen(f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                        {base_name}.orc f{col_idx} {dtype} none \
                            0 0 one").read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['format'] = fmt
                    output_stats['i'] = i
                    output_stats['col_idx'] = col_idx
                    output_stats['num_cols'] = num_cols
                    output_stats['num_rows'] = num_rows
                    parse_output(output_stats)
collect_results()


/mnt/OpenFormat/python/wide_table


In [5]:
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('projection_exp_v2', timestamp))

0